
# <center>Escuela Militar de Ingeniería</center>
<center>Ingeniería de Sistemas</center>
<center><img src="https://i.servimg.com/u/f73/20/03/60/99/logo_i12.jpg" width="300"></center>
<center>Estudiante: Zalles Merino Izai Alejandro</center>
<center>Código SAGA: A27006-7</center>

## <center>Docente: Ing. Angela Santos</center>
## Fecha: 11 de Abril de 2025


In [ ]:

#@title Convertir Notebook (.ipynb) a HTML en Google Colab

# Subir archivo ipynb
from google.colab import files
archivosSubidos <- files.upload()

# Convertir archivo ipynb a HTML
import subprocess
nombreArchivo <- list(archivosSubidos.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", nombreArchivo, "--to", "html"])

# Descargar el archivo HTML generado
files.download(nombreArchivo[:-5] + "html")
